In [ ]:
import networkx as nx
import pandas as pd

# 1. Carica il file GEXF appena creato
# Assicurati che il nome file corrisponda a quello salvato nel main
# filename = "dataset_indignazione.gexf" 
filename = "dataset_user.gexf" 

try:
    G = nx.read_gexf(filename)
    print(f"✅ File caricato! Nodi: {G.number_of_nodes()}, Archi: {G.number_of_edges()}")
except FileNotFoundError:
    print(f"❌ File {filename} non trovato.")

# --- VISUALIZZAZIONE ARCHI (Approccio C) ---
data_archi = []

# Iteriamo su tutti gli archi per estrarre i dati
for source, target, attributes in G.edges(data=True):
    row = {
        'Chi Risponde (Source)': source,
        'A Chi (Target)': target,
        'MAGNETE (Post Originale)': attributes.get('trigger_text', ''), # <--- La causa
        'REAZIONE (Risposta)': attributes.get('reply_content', '')      # <--- L'effetto
    }
    data_archi.append(row)

df_edges = pd.DataFrame(data_archi)

# --- VISUALIZZAZIONE ---
print("\n--- TEST APPROCCIO C: Causa ed Effetto ---")
# Opzione per leggere tutto il testo
pd.set_option('display.max_colwidth', None) 

# Mostriamo le prime 5 righe
# Dovresti vedere che per lo stesso Target, il "Magnete" è identico
display(df_edges.head(5))

# Verifica rapida: Contiamo quanti archi hanno lo stesso "Magnete"
# Questo simula la tua futura analisi di popolarità
if not df_edges.empty:
    print("\n--- TOP MAGNETI (Post che hanno attirato più risposte) ---")
    top_magneti = df_edges['MAGNETE (Post Originale)'].value_counts().head(3)
    for text, count in top_magneti.items():
        print(f"\n🔥 {count} Risposte al post che dice:\n'{text[:100]}...'")
        
        
        
# --- VISUALIZZAZIONE NODI (Gli Utenti) ---
data_nodi = []

# Iteriamo su tutti i nodi per estrarre gli attributi anagrafici
for node_id, attributes in G.nodes(data=True):
    row = {
        'Utente': node_id,
        'Followers': attributes.get('followers', 0),
        'Posts Totali': attributes.get('posts', 0)
    }
    data_nodi.append(row)

df_nodes = pd.DataFrame(data_nodi)

# --- VISUALIZZAZIONE ---
print("\n--- DATI DEI NODI (Utenti) ---")
# Mostriamo i primi 5 utenti ordinati per numero di followers
display(df_nodes.sort_values(by='Followers', ascending=False).head(5))

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Inizializziamo l'analizzatore
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if not text: return 0
    # Il punteggio 'compound' va da -1 (odio/rabbia) a +1 (gioia/supporto)
    return analyzer.polarity_scores(text)['compound']

# 2. Applichiamo l'analisi al DataFrame degli archi
print("🧠 Analisi del sentiment in corso...")
df_edges['sentiment_magnete'] = df_edges['MAGNETE (Post Originale)'].apply(get_sentiment)
df_edges['sentiment_reazione'] = df_edges['REAZIONE (Risposta)'].apply(get_sentiment)

# sort df_edges by MAGNETE (Post Originale)
df_edges = df_edges.sort_values(by='MAGNETE (Post Originale)')

# 3. AGGREGAZIONE: Calcoliamo la popolarità per ogni post originale
# Raggruppiamo per il testo del magnete e contiamo quanti archi (risposte) ha attirato
df_popolarita = df_edges.groupby('MAGNETE (Post Originale)').agg({
    'sentiment_magnete': 'first', # Il sentiment del post è lo stesso per tutte le sue risposte
    'Chi Risponde (Source)': 'count' # Contiamo le risposte
}).rename(columns={'Chi Risponde (Source)': 'numero_risposte'})

# 4. VISUALIZZAZIONE: Correlazione Sentiment vs Popolarità
plt.figure(figsize=(10, 6))
sns.regplot(data=df_popolarita, x='sentiment_magnete', y='numero_risposte', 
            scatter_kws={'alpha':0.5}, line_kws={'color':'red'})

plt.title("Correlazione: Sentiment del Post vs Numero di Risposte")
plt.xlabel("Sentiment (Negativo < 0 > Positivo)")
plt.ylabel("Numero di Risposte (Popolarità)")
plt.grid(True, alpha=0.3)
plt.show()

# Vediamo i 5 post più negativi che hanno generato traffico
print("\n🔥 TOP 5 POST NEGATIVI (Potenziale Indignazione):")
display(df_popolarita.sort_values(by='sentiment_magnete').head(5))